In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.applications import MobileNetV2

In [ ]:
base_model = MobileNetV2(weights='imagenet',include_top=False)
for layer in base_model.layers:
    layer.trainable=False
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, None, None, 3 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, None, None, 3 128         Conv1[0][0]                      
_______________________________________________________________________________

In [ ]:
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D


x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(128)(x)
X=Dropout(0.25)(x)
x=Dense(128)(x)
preds=Dense(42 ,activation='softmax')(x) #final layer with softmax activation for N classes

model=Model(inputs=base_model.input,outputs=preds) #specify the inputs and outputs
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, None, None, 3 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, None, None, 3 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [ ]:
import os
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_dir = '/content/drive/My Drive/Shopee data/new_train/'
validation_dir = '/content/drive/My Drive/Shopee data/new_test/'

image_generator = ImageDataGenerator(
      rescale=1./255,
      # validation_split=0.2
      # rotation_range=40,
      # width_shift_range=0.2,
      # height_shift_rang0.e=2,
      # shear_range=0.2,
      # zoom_range=0.2,
      # horizontal_flip=True,
      # fill_mode='nearest'
      )   

train_dataset = image_generator.flow_from_directory(
        train_dir,  
        target_size=(224, 224), 
        batch_size=100,
        class_mode="categorical",
        shuffle=True)

validation_dataset = image_generator.flow_from_directory(batch_size=24,
                                                 directory=validation_dir,
                                                 shuffle=True,
                                                 target_size=(224, 224), 
                                                 class_mode='categorical')

Found 18900 images belonging to 42 classes.
Found 4536 images belonging to 42 classes.


In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('/content/drive/My Drive/Shopee data/MobileNetV2_save.h5', verbose=1, save_best_only=True)
callbacks = [tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss'),
             tf.keras.callbacks.TensorBoard(log_dir='/content/drive/My Drive/MobileNetV2_log'),
             checkpoint]

model.compile(loss= tf.keras.losses.CategoricalCrossentropy(),
              optimizer=Adam(learning_rate=0.002),
              metrics=['accuracy'])

history = model.fit(
      train_dataset,
      callbacks=callbacks,
      validation_data=validation_dataset,  
      epochs=20,
      verbose=1)

Epoch 1/20
189/189 [==============================] - ETA: 0s - loss: 1.7634 - accuracy: 0.5337  
Epoch 00001: val_loss improved from inf to 1.55950, saving model to /content/drive/My Drive/Shopee data/MobileNetV2_save.h5
189/189 [==============================] - 15693s 83s/step - loss: 1.7634 - accuracy: 0.5337 - val_loss: 1.5595 - val_accuracy: 0.5899
Epoch 2/20
189/189 [==============================] - ETA: 0s - loss: 1.2970 - accuracy: 0.6465
Epoch 00002: val_loss improved from 1.55950 to 1.51086, saving model to /content/drive/My Drive/Shopee data/MobileNetV2_save.h5
189/189 [==============================] - 248s 1s/step - loss: 1.2970 - accuracy: 0.6465 - val_loss: 1.5109 - val_accuracy: 0.6096
Epoch 3/20
189/189 [==============================] - ETA: 0s - loss: 1.1445 - accuracy: 0.6785
Epoch 00003: val_loss did not improve from 1.51086
189/189 [==============================] - 244s 1s/step - loss: 1.1445 - accuracy: 0.6785 - val_loss: 1.6242 - val_accuracy: 0.5941
Epoch 4/